# Wege & Zielpersonen & Haushalte

## 1. Import

### 1.1 Bib

In [210]:
#import pandas as pd
import numpy as np
import pandas as pd



### 1.2 Data

In [211]:
wege = pd.read_csv('Data/wege.csv', sep=';', encoding='latin1')
zielpersonen = pd.read_csv('Data/zielpersonen.csv', sep=';', encoding='latin1')
haushalte = pd.read_csv('Data/haushalte.csv', sep=';', encoding='latin1')



/var/folders/40/mpbcjvhd0j52jzpxtm8yrg6r0000gn/T/ipykernel_45782/2796337158.py:3: DtypeWarning: Columns (53,66) have mixed types. Specify dtype option on import or set low_memory=False.
  haushalte = pd.read_csv('Data/haushalte.csv', sep=';', encoding='latin1')


## 2. Bereinigung Daten

In [212]:

weg=wege[['HHNR', 'WEGNR', 'S_X', 'S_Y', 'Z_X', 'Z_Y', 'f51700_weg', 'dauer2', 'wzweck1']]
weg.rename(columns={'HHNR': 'HHNR', 'WEGNR': 'Wegnummer', 'S_X': 'Start_X-Koordinate', 'S_Y': 'Start_Y-Koordinate', 'Z_X': 'Ziel_X-Koordinate', 'Z_Y': 'Ziel_Y-Koordinate', 'f51700_weg': 'Freizeitaktivität', 'dauer2': 'Dauer in Minuten', 'wzweck1': 'Wegzweck'}, inplace=True)
weg.head()

/var/folders/40/mpbcjvhd0j52jzpxtm8yrg6r0000gn/T/ipykernel_45782/2561939426.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weg.rename(columns={'HHNR': 'HHNR', 'WEGNR': 'Wegnummer', 'S_X': 'Start_X-Koordinate', 'S_Y': 'Start_Y-Koordinate', 'Z_X': 'Ziel_X-Koordinate', 'Z_Y': 'Ziel_Y-Koordinate', 'f51700_weg': 'Freizeitaktivität', 'dauer2': 'Dauer in Minuten', 'wzweck1': 'Wegzweck'}, inplace=True)


,HHNR,Wegnummer,Start_X-Koordinate,Start_Y-Koordinate,Ziel_X-Koordinate,Ziel_Y-Koordinate,Freizeitaktivität,Dauer in Minuten,Wegzweck
0,110001,1,6.559870,46.549366,6.564175,46.548909,-99,10,5.0
1,110001,2,6.564175,46.548909,6.573270,46.553510,-99,5,4.0
2,110001,3,6.573270,46.553510,6.559870,46.549366,-99,10,4.0
3,110001,4,6.559870,46.549366,6.552598,46.568205,2,10,8.0
4,110001,5,6.552598,46.568205,6.554009,46.540095,-99,15,5.0


In [213]:
#länge der Tabelle
len(weg)

151188

In [214]:
#zähle unique values in der Spalte WEgzweck
weg['Wegzweck'].value_counts()

Wegzweck
 8.0     55751
 2.0     30035
 4.0     27756
 3.0     12864
 5.0     10314
 9.0      5453
 10.0     4288
 11.0     1358
 12.0     1126
 6.0      1097
 7.0      1006
-99.0       93
-98.0       36
-97.0        9
Name: count, dtype: int64

In [215]:
zielpersonen.head()
zielp=zielpersonen[['HHNR', 'alter', 'gesl', 'f20400a','f20400b', 'f20400c', 'f20400d', 'f40120', 'f81200', 'f50100a','f42100a', 'f42100b', 'f42100c', 'f42100d', 'f42100e', 'zivil', 'ESTATUS', 'T_FREIZEIT']]
zielp.rename(columns={'HHNR': 'HHNR', 'alter': 'Alter', 'gesl': 'Geschlecht', 'f20400a': 'Führerausweis PW', 'f20400b': 'Führerausweis Motorräder', 'f20400c': 'Lernfahrausweis PW', 'f20400d': 'Lernfahrausweis Motorräder', 'f40120': 'Ausbildungsniveau', 'f81200': 'Arbeitszeitmodelle', 'f50100a': 'Wetter',  'f42100a': 'Velo-Besitzer', 'f42100b': 'Mofa-Besitzer', 'f42100c': 'Kleinmotorrad-Besitzer', 'f42100d': 'Motorrad-Besitzer', 'f42100e': 'Auto-Besitzer', 'zivil': 'Familienstand', 'ESTATUS': 'Erwerbsstatus', 'T_FREIZEIT': 'Tägliche Freizeitwege'}, inplace=True)
zielp.head()

/var/folders/40/mpbcjvhd0j52jzpxtm8yrg6r0000gn/T/ipykernel_45782/2600206432.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zielp.rename(columns={'HHNR': 'HHNR', 'alter': 'Alter', 'gesl': 'Geschlecht', 'f20400a': 'Führerausweis PW', 'f20400b': 'Führerausweis Motorräder', 'f20400c': 'Lernfahrausweis PW', 'f20400d': 'Lernfahrausweis Motorräder', 'f40120': 'Ausbildungsniveau', 'f81200': 'Arbeitszeitmodelle', 'f50100a': 'Wetter',  'f42100a': 'Velo-Besitzer', 'f42100b': 'Mofa-Besitzer', 'f42100c': 'Kleinmotorrad-Besitzer', 'f42100d': 'Motorrad-Besitzer', 'f42100e': 'Auto-Besitzer', 'zivil': 'Familienstand', 'ESTATUS': 'Erwerbsstatus', 'T_FREIZEIT': 'Tägliche Freizeitwege'}, inplace=True)


,HHNR,Alter,Geschlecht,Führerausweis PW,Führerausweis Motorräder,Lernfahrausweis PW,Lernfahrausweis Motorräder,Ausbildungsniveau,Arbeitszeitmodelle,Wetter,Velo-Besitzer,Mofa-Besitzer,Kleinmotorrad-Besitzer,Motorrad-Besitzer,Auto-Besitzer,Familienstand,Erwerbsstatus,Tägliche Freizeitwege
0,110001,57,1,1,1,-99,-99,4,-99,1,3,3,3,1,1,4,3,3
1,110008,57,1,1,1,-99,-99,2,-99,1,3,3,3,1,3,2,5,2
2,110011,16,2,-99,-99,-99,-99,2,-99,5,1,3,3,-99,-99,1,4,1
3,110024,17,2,-99,-99,2,2,2,-99,6,1,2,2,-99,-99,1,4,0
4,110026,53,2,1,1,-99,-99,4,-99,1,1,3,3,1,2,1,8,3


In [216]:
len(zielp)

55018

In [217]:
# dataset haushalte auf W_X, W_Y, W_ORT, W_PLZ, W_KANTON
haushalte=haushalte[['HHNR', 'W_X', 'W_Y', 'W_ORT', 'W_PLZ', 'W_KANTON']]
haushalte.head()

,HHNR,W_X,W_Y,W_ORT,W_PLZ,W_KANTON
0,110001,6.559870,46.549366,Bussigny,1030,22
1,110008,6.635652,46.785591,Yverdon-les-Bains,1400,22
2,110011,9.112485,47.451677,Henau,9247,17
3,110024,7.499629,46.897868,Belp,3123,2
4,110026,8.658613,47.724048,Schaffhausen,8207,14


In [218]:
# Dictionary zur Umbenennung der Spalten
rename_mapping = {
    "W_X": "Wohnort X-Koordinate",
    "W_Y": "Wohnort Y-Koordinate",
    "W_PLZ": "Wohnort: PLZ",
    "W_KANTON": "Wohnort: Kanton"
}

# Umbenennen der Spalten im DataFrame
haushalte = haushalte.rename(columns=rename_mapping)

Die Datensätze beinhalten keine Null-Werte und keine Duplikate

## 3. Merge

In [219]:
# Right Join der DataFrames auf Basis der Spalte 'HHNR'
merged_data = weg.merge(zielp, on='HHNR', how='left')


In [220]:
# Finde alle Wege, denen keine Zielperson zugeordnet ist
no_zielperson = merged_data[merged_data['Alter'].isna()]  # 'alter' ist ein Beispiel; ersetze es durch eine Spalte, die nur in 'zielpersonen' existiert

# Gib die HHNR-Werte aus, die keine Zielperson zugeordnet haben
nicht_zugeordnete_wege = no_zielperson['HHNR'].unique()  # Gibt die eindeutigen HHNR ohne Zuordnung

print("Anzahl der Wege ohne zugeordnete Zielperson:", len(nicht_zugeordnete_wege))
print("HHNR-Werte ohne zugeordnete Zielperson:", nicht_zugeordnete_wege)

Anzahl der Wege ohne zugeordnete Zielperson: 0
HHNR-Werte ohne zugeordnete Zielperson: []


In [221]:
#haushalte zu merged_data mergen über die Spalte HHNR
merged_data = merged_data.merge(haushalte, on='HHNR', how='left')

In [222]:
len(merged_data)

151188

Dataset verfügt noch über 151'188 Einträge

In [223]:
merged_data.head()

,HHNR,Wegnummer,Start_X-Koordinate,Start_Y-Koordinate,Ziel_X-Koordinate,Ziel_Y-Koordinate,Freizeitaktivität,Dauer in Minuten,Wegzweck,Alter,...,Motorrad-Besitzer,Auto-Besitzer,Familienstand,Erwerbsstatus,Tägliche Freizeitwege,Wohnort X-Koordinate,Wohnort Y-Koordinate,W_ORT,Wohnort: PLZ,Wohnort: Kanton
0,110001,1,6.559870,46.549366,6.564175,46.548909,-99,10,5.0,57,...,1,1,4,3,3,6.55987,46.549366,Bussigny,1030,22
1,110001,2,6.564175,46.548909,6.573270,46.553510,-99,5,4.0,57,...,1,1,4,3,3,6.55987,46.549366,Bussigny,1030,22
2,110001,3,6.573270,46.553510,6.559870,46.549366,-99,10,4.0,57,...,1,1,4,3,3,6.55987,46.549366,Bussigny,1030,22
3,110001,4,6.559870,46.549366,6.552598,46.568205,2,10,8.0,57,...,1,1,4,3,3,6.55987,46.549366,Bussigny,1030,22
4,110001,5,6.552598,46.568205,6.554009,46.540095,-99,15,5.0,57,...,1,1,4,3,3,6.55987,46.549366,Bussigny,1030,22


In [224]:
merged_data.columns

Index(['HHNR', 'Wegnummer', 'Start_X-Koordinate', 'Start_Y-Koordinate',
       'Ziel_X-Koordinate', 'Ziel_Y-Koordinate', 'Freizeitaktivität',
       'Dauer in Minuten', 'Wegzweck', 'Alter', 'Geschlecht',
       'Führerausweis PW', 'Führerausweis Motorräder', 'Lernfahrausweis PW',
       'Lernfahrausweis Motorräder', 'Ausbildungsniveau', 'Arbeitszeitmodelle',
       'Wetter', 'Velo-Besitzer', 'Mofa-Besitzer', 'Kleinmotorrad-Besitzer',
       'Motorrad-Besitzer', 'Auto-Besitzer', 'Familienstand', 'Erwerbsstatus',
       'Tägliche Freizeitwege', 'Wohnort X-Koordinate', 'Wohnort Y-Koordinate',
       'W_ORT', 'Wohnort: PLZ', 'Wohnort: Kanton'],
      dtype='object')

## 4. Filter


Wegzwecke Filter auf 8 setzen für Freizeitwege

In [225]:
merged_data['Wegzweck'].unique()

array([  5.,   4.,   8.,   2.,  11.,   3.,   9.,   7.,  10.,  12.,   6.,
       -99., -98., -97.,  nan])

In [226]:
#Filtere komplette Tabelle auf Wegzwecke = 8
merged_data_leisure = merged_data[merged_data['Wegzweck'] == 8]

In [227]:
len(merged_data_leisure)

55751

Dataset verfügt noch über 55'751 Einträge

Art der Freizeitaktivität kategorisieren:

- -99: WZWECK1 NE 8
- -98: Keine Antwort
- -97: weiss nicht

- 18: Anderes - manuell Aufnehmen
- 22: mehrere Aktivitäten

Alles zu unbekannt


In [228]:
# Ersetzen der spezifischen Werte -99, -98, -97, 18 und 22 durch "unbekannt"
merged_data_leisure.loc[merged_data_leisure['Freizeitaktivität'].isin([-99, -98, -97, 18, 22]), 'Freizeitaktivität'] = 'unbekannt'


/var/folders/40/mpbcjvhd0j52jzpxtm8yrg6r0000gn/T/ipykernel_45782/2156852050.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'unbekannt' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  merged_data_leisure.loc[merged_data_leisure['Freizeitaktivität'].isin([-99, -98, -97, 18, 22]), 'Freizeitaktivität'] = 'unbekannt'


Mofa- und Kleinmotorradbesitzer zusammen nehmen und nach Mofa bennenen

In [229]:
#Spalte Mofa-Besitzer und Kleinmotorrad-Besitzer zusammenfassen
merged_data_leisure['Mofa'] = merged_data_leisure['Mofa-Besitzer'] + merged_data_leisure['Kleinmotorrad-Besitzer']

/var/folders/40/mpbcjvhd0j52jzpxtm8yrg6r0000gn/T/ipykernel_45782/415521471.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_data_leisure['Mofa'] = merged_data_leisure['Mofa-Besitzer'] + merged_data_leisure['Kleinmotorrad-Besitzer']


Freizeitaktivitäten; Nummern ausschreiben

    1: "Besuche (Verwandte, Bekannte, Freunde)",
    2: "Gastronomiebesuch (Restaurant, Bar, Café,)",
    3: "Aktiver Sport (Fussball, Jogging)",
    4: "Wanderung",
    5: "Velofahrt",
    6: "Passiver Sport (Matchbesuch)",
    7: "Nicht-sportliche Aussenaktivität (z.B. Spazieren mit dem Hund)",
    8: "Medizin/Wellness/Fitness",
    9: "Kulturveranstaltungen, Freizeitanlagen (z.B. Vergnügungsparks)",
    10: "Unbezahlte Arbeit",
    11: "Vereinstätigkeit",
    12: "Ausflug, Ferien",
    13: "Religion (Kirche, Friedhof, Pilgerfahrt)",
    14: "Häusliche Freizeitaktivitäten auswärts",
    15: "Essen ohne Gastronomiebesuch (Pick-Nick, Grillparty etc.)",
    16: "Einkaufsbummel / Shopping",
    17: "Rundreise (Auto-/Motorrad-/Zugsfahrt etc.)"

In [230]:
# Mapping-Liste für die neuen Bezeichnungen
freizeitaktivitaeten_mapping = {
    1: "Besuche",
    2: "Gastronomiebesuch",
    3: "Aktiver Sport",
    4: "Wanderung",
    5: "Velofahrt",
    6: "Passiver Sport",
    7: "Nicht-sportliche Aussenaktivität",
    8: "Medizin/Wellness/Fitness",
    9: "Kulturveranstaltungen, Freizeitanlagen",
    10: "Unbezahlte Arbeit",
    11: "Vereinstätigkeit",
    12: "Ausflug, Ferien",
    13: "Religion",
    14: "Häusliche Freizeitaktivitäten auswärts",
    15: "Essen ohne Gastronomiebesuch",
    16: "Einkaufsbummel / Shopping",
    17: "Rundreise",
}

# Umbenennen der Werte in der Spalte 'Freizeitaktivität' anhand der Mapping-Liste
merged_data_leisure["Freizeitaktivität"] = merged_data_leisure["Freizeitaktivität"].map(freizeitaktivitaeten_mapping)


/var/folders/40/mpbcjvhd0j52jzpxtm8yrg6r0000gn/T/ipykernel_45782/1724883842.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_data_leisure["Freizeitaktivität"] = merged_data_leisure["Freizeitaktivität"].map(freizeitaktivitaeten_mapping)


In [231]:
# Mapping-Dictionary für die Kantonsnummern
kanton_mapping = {
    -97: "unbekannt",
    1: "Zürich",
    2: "Bern",
    3: "Luzern",
    4: "Uri",
    5: "Schwyz",
    6: "Obwalden",
    7: "Nidwalden",
    8: "Glarus",
    9: "Zug",
    10: "Fribourg",
    11: "Solothurn",
    12: "Basel-Stadt",
    13: "Basel-Land",
    14: "Schaffhausen",
    15: "Appenzell Ausserrhoden",
    16: "Appenzell Innerrhoden",
    17: "St. Gallen",
    18: "Graubünden",
    19: "Aargau",
    20: "Thurgau",
    21: "Ticino",
    22: "Vaud",
    23: "Valais",
    24: "Neuchâtel",
    25: "Genève",
    26: "Jura"
}

# Umbenennen der Werte in der Spalte 'Wohnort: Kanton' anhand der Mapping-Liste
merged_data_leisure["Wohnort: Kanton"] = merged_data_leisure["Wohnort: Kanton"].map(kanton_mapping)



/var/folders/40/mpbcjvhd0j52jzpxtm8yrg6r0000gn/T/ipykernel_45782/4223841432.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_data_leisure["Wohnort: Kanton"] = merged_data_leisure["Wohnort: Kanton"].map(kanton_mapping)


In [232]:
len(merged_data_leisure)

55751

In [233]:
merged_data_leisure.head()

,HHNR,Wegnummer,Start_X-Koordinate,Start_Y-Koordinate,Ziel_X-Koordinate,Ziel_Y-Koordinate,Freizeitaktivität,Dauer in Minuten,Wegzweck,Alter,...,Auto-Besitzer,Familienstand,Erwerbsstatus,Tägliche Freizeitwege,Wohnort X-Koordinate,Wohnort Y-Koordinate,W_ORT,Wohnort: PLZ,Wohnort: Kanton,Mofa
3,110001,4,6.559870,46.549366,6.552598,46.568205,Gastronomiebesuch,10,8.0,57,...,1,4,3,3,6.559870,46.549366,Bussigny,1030,Vaud,6
5,110001,6,6.554009,46.540095,6.554009,46.540095,Einkaufsbummel / Shopping,5,8.0,57,...,1,4,3,3,6.559870,46.549366,Bussigny,1030,Vaud,6
6,110001,7,6.554009,46.540095,6.559870,46.549366,NaN,10,8.0,57,...,1,4,3,3,6.559870,46.549366,Bussigny,1030,Vaud,6
7,110008,1,6.635652,46.785591,6.635652,46.785591,Nicht-sportliche Aussenaktivität,120,8.0,57,...,3,2,5,2,6.635652,46.785591,Yverdon-les-Bains,1400,Vaud,6
10,110008,4,6.635652,46.785591,6.635652,46.785591,Aktiver Sport,120,8.0,57,...,3,2,5,2,6.635652,46.785591,Yverdon-les-Bains,1400,Vaud,6


Dataset verfügt noch über 55'188 Einträge